# 🤖 AI Agents Bootcamp - Day 1
## Your First AI Agent: Company Research

**Welcome!** This notebook accompanies [Day 1 of the AI Agents series](https://teodoracoach.substack.com/).

By the end of this notebook, you'll understand:
- The 5 building blocks of AI agents
- How to build a simple research agent
- The difference between agents and regular AI

---

**⚠️ Before you start:** You'll need an OpenAI API key.
- Get one at [platform.openai.com](https://platform.openai.com)
- This notebook costs ~$0.05 to run

## 1️⃣ Setup

In [ ]:
# Install required packages
!pip install -q openai python-dotenv

In [ ]:
# Import libraries
import os
from datetime import datetime
from openai import OpenAI
from google.colab import userdata

print("✅ Libraries imported successfully!")

In [ ]:
# Set up your API key
# Option 1: Use Colab Secrets (recommended)
# Go to 🔑 icon in left sidebar → Add secret named 'OPENAI_API_KEY'

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab Secrets")
except:
    # Option 2: Paste directly (less secure, but works)
    OPENAI_API_KEY = "sk-your-key-here"  # ⚠️ Replace with your actual key
    print("⚠️ Using hardcoded API key - consider using Colab Secrets instead")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)
print("✅ OpenAI client initialized!")

## 2️⃣ The 5 Building Blocks of AI Agents

Every AI agent has these components:

| Block | Description | In Our Agent |
|-------|-------------|---------------|
| 🎯 **Goal** | What to accomplish | Research a company |
| 🧠 **Reasoning** | LLM that plans | GPT-4 |
| 🔧 **Tools** | Capabilities | LLM knowledge (Day 2: web search) |
| 💾 **Memory** | Store context | Dictionary with results |
| 🔄 **Feedback** | Self-evaluate | Validation check |

## 3️⃣ Build the Agent

In [ ]:
# ==========================================================
# CONFIGURATION
# ==========================================================

RESEARCH_PROMPT = """You are a company research agent helping someone prepare for a job interview.

For each company, provide:

## 1. Company Overview
- What they do, when founded, headquarters
- Key leadership

## 2. Products & Services
- Main offerings and target customers

## 3. Culture & Values
- Mission, values, work environment

## 4. Recent Developments
- News from last 12-18 months

## 5. Interview Tips
- What they look for, how to stand out

Be concise but thorough. Use bullet points."""

VALIDATION_PROMPT = """Check if this research covers ALL sections:
1. Company Overview
2. Products & Services
3. Culture & Values
4. Recent Developments
5. Interview Tips

Respond with ONLY:
- "COMPLETE" if all covered
- "INCOMPLETE: [what's missing]" if not"""

print("✅ Configuration set!")

In [ ]:
def research_company(company_name: str) -> dict:
    """
    AI Agent that researches a company.

    Demonstrates all 5 building blocks:
    1. Goal - Research the company
    2. Reasoning - GPT-4 gathers info
    3. Tools - LLM knowledge
    4. Memory - Store results
    5. Feedback - Validate output
    """

    print(f"\n{'='*50}")
    print(f"🤖 AGENT ACTIVATED")
    print(f"{'='*50}")
    print(f"📎 Goal: Research '{company_name}'")
    print(f"{'='*50}\n")

    # =============================================
    # BUILDING BLOCK 1: GOAL
    # =============================================
    goal = f"Research {company_name} for interview prep"

    # =============================================
    # BUILDING BLOCK 2 & 3: REASONING + TOOLS
    # =============================================
    print("📊 Step 1/3: Gathering information...")

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": RESEARCH_PROMPT},
            {"role": "user", "content": f"Research: {company_name}"}
        ],
        temperature=0.7
    )

    research_output = response.choices[0].message.content

    # =============================================
    # BUILDING BLOCK 4: MEMORY
    # =============================================
    memory = {
        "company": company_name,
        "goal": goal,
        "research": research_output,
        "timestamp": datetime.now().isoformat(),
        "status": "pending"
    }

    # =============================================
    # BUILDING BLOCK 5: FEEDBACK LOOP
    # =============================================
    print("✅ Step 2/3: Validating completeness...")

    validation = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": VALIDATION_PROMPT},
            {"role": "user", "content": research_output}
        ],
        temperature=0
    )

    validation_result = validation.choices[0].message.content

    if "INCOMPLETE" in validation_result:
        memory["status"] = "needs_review"
        print(f"⚠️ {validation_result}")
    else:
        memory["status"] = "complete"
        print("✅ Research validated as complete!")

    print("📝 Step 3/3: Preparing output...")

    return memory

print("✅ Agent function defined!")

## 4️⃣ Run the Agent!

In [ ]:
# Choose a company to research
# Try: "Stripe", "Anthropic", "Airbnb", "Notion", or any company!

COMPANY = "Stripe"  # 👈 Change this!

# Run the agent
results = research_company(COMPANY)

In [ ]:
# Display the research report
from IPython.display import Markdown, display

print("\n" + "="*60)
print(f"📋 RESEARCH REPORT: {results['company'].upper()}")
print(f"Status: {results['status']}")
print("="*60 + "\n")

display(Markdown(results['research']))

In [ ]:
# Save to file (optional)
filename = f"{results['company'].lower().replace(' ', '_')}_research.md"

with open(filename, 'w') as f:
    f.write(f"# Research Report: {results['company']}\n\n")
    f.write(f"Generated: {results['timestamp']}\n\n")
    f.write(f"Status: {results['status']}\n\n---\n\n")
    f.write(results['research'])

print(f"💾 Saved to: {filename}")

# Download the file
from google.colab import files
files.download(filename)

## 5️⃣ Experiment!

Try these modifications:

1. **Change the company** - Edit `COMPANY` above
2. **Modify the research focus** - Edit `RESEARCH_PROMPT`
3. **Add new sections** - What else would help in interviews?

---

## 🎯 Key Takeaways

You just built an AI agent with:
- ✅ **Goal**: Research a company
- ✅ **Reasoning**: GPT-4 planning and executing
- ✅ **Tools**: LLM knowledge (real tools in Day 2)
- ✅ **Memory**: Stored results in a dictionary
- ✅ **Feedback**: Validated output completeness

---

## ➡️ Next Steps

**Day 2**: Multi-agent systems with CrewAI
- Multiple agents working together
- Real web search tools
- Interview preparation system

**Subscribe**: [teodoracoach.substack.com](https://teodoracoach.substack.com/)

**Full code**: [GitHub repo](https://github.com/your-username/ai-agents-bootcamp)